In [6]:
import os
from dotenv import load_dotenv
import requests

from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import tool

load_dotenv()
WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")


@tool
def get_current_weather(city: str) -> str:
    """
    برای گرفتن آب و هوای فعلی یک شهر مشخص از این ابزار استفاده کن.
    ورودی باید نام شهر به انگلیسی یا فارسی باشد، مثلا: 'Tehran' یا 'اصفهان'.
    """
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        'q': city,
        'appid': WEATHER_API_KEY,
        'units': 'metric',
        'lang': 'fa'
    }
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()

        description = data['weather'][0]['description']
        temp = data['main']['temp']
        humidity = data['main']['humidity']
        wind_speed = data['wind']['speed']

        return f"آب و هوای فعلی در شهر {city}: {description}، دما {temp}°C، رطوبت {humidity}% و سرعت باد {wind_speed} متر بر ثانیه است."

    except requests.exceptions.HTTPError as http_err:
        if response.status_code == 404:
            return f"متاسفانه نتوانستم شهری به نام '{city}' را پیدا کنم. لطفاً نام شهر را به درستی وارد کنید."
        return f"HTTP error: {http_err}"


tools = [get_current_weather]

SYSTEM_MESSAGE = """
تو یک دستیار مفید هستی که تنها وظیفه‌ات ارائه اطلاعات آب و هوای فعلی یک شهر مشخص است.
برای پاسخ به سوالات کاربر، حتماً باید از ابزار `get_current_weather` استفاده کنی.
پاسخ نهایی را به صورت کامل و روان به زبان فارسی به کاربر ارائه بده.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_MESSAGE),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_tools_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)


def ask_agent(question):
    print(f"user question: {question}")
    response = agent_executor.invoke({"input": question})
    print("Agent response:")
    print(response["output"])

    print("\n\n========================================")


ask_agent("هوای تهران چطوره؟")



Agent آب و هوا در حال اجرا با سوالات تعریف شده...
🤔 سوال شما: هوای تهران چطوره؟


> Entering new AgentExecutor chain...

Invoking: `get_current_weather` with `{'city': 'Tehran'}`


  [ابزار صدا زده شد: در حال دریافت آب و هوای شهر Tehran]
آب و هوای فعلی در شهر Tehran: کمی ابری، دما 25.4°C، رطوبت 14% و سرعت باد 7.2 متر بر ثانیه است.آب و هوای فعلی در تهران کمی ابری است با دمای 25.4 درجه سانتیگراد، رطوبت 14 درصد و سرعت باد 7.2 متر بر ثانیه.

> Finished chain.

✅ پاسخ Agent:
آب و هوای فعلی در تهران کمی ابری است با دمای 25.4 درجه سانتیگراد، رطوبت 14 درصد و سرعت باد 7.2 متر بر ثانیه.

🤔 سوال شما: وضعیت آب و هوای شهر رشت چگونه است؟


> Entering new AgentExecutor chain...

Invoking: `get_current_weather` with `{'city': 'Rasht'}`


  [ابزار صدا زده شد: در حال دریافت آب و هوای شهر Rasht]
آب و هوای فعلی در شهر Rasht: کمی ابری، دما 27.9°C، رطوبت 34% و سرعت باد 3.09 متر بر ثانیه است.آب و هوای فعلی در شهر رشت کمی ابری است با دمای 27.9 درجه سانتی‌گراد، رطوبت 34 درصد و سرعت باد 3.09 متر بر ثانیه.

> Fi

In [7]:
ask_agent("وضعیت آب و هوای شهر رشت چگونه است؟")

🤔 سوال شما: وضعیت آب و هوای شهر رشت چگونه است؟


> Entering new AgentExecutor chain...

Invoking: `get_current_weather` with `{'city': 'Rasht'}`


  [ابزار صدا زده شد: در حال دریافت آب و هوای شهر Rasht]
آب و هوای فعلی در شهر Rasht: کمی ابری، دما 27.9°C، رطوبت 34% و سرعت باد 3.09 متر بر ثانیه است.آب و هوای فعلی در شهر رشت کمی ابری است با دمای 27.9 درجه سانتی‌گراد، رطوبت 34 درصد و سرعت باد 3.09 متر بر ثانیه.

> Finished chain.

✅ پاسخ Agent:
آب و هوای فعلی در شهر رشت کمی ابری است با دمای 27.9 درجه سانتی‌گراد، رطوبت 34 درصد و سرعت باد 3.09 متر بر ثانیه.



In [8]:
ask_agent("امروز تو لندن بارون اومد؟")


🤔 سوال شما: امروز تو لندن بارون اومد؟


> Entering new AgentExecutor chain...

Invoking: `get_current_weather` with `{'city': 'London'}`


  [ابزار صدا زده شد: در حال دریافت آب و هوای شهر London]
آب و هوای فعلی در شهر London: کمی ابری، دما 15.78°C، رطوبت 56% و سرعت باد 6.17 متر بر ثانیه است.امروز در لندن باران نیامده است و هوا کمی ابری بوده است.

> Finished chain.

✅ پاسخ Agent:
امروز در لندن باران نیامده است و هوا کمی ابری بوده است.

